In [14]:
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.layers import Dense, Input, LSTM, Embedding, Dropout, Activation, Conv1D, GRU, CuDNNGRU, CuDNNLSTM, BatchNormalization
from keras.layers import Bidirectional, GlobalMaxPool1D, MaxPooling1D, Add, Flatten
from keras.layers import GlobalAveragePooling1D, GlobalMaxPooling1D, concatenate, SpatialDropout1D
from keras.models import Model, load_model
from keras import initializers, regularizers, constraints, optimizers, layers, callbacks
from keras import backend as K
from keras.engine import InputSpec, Layer
from keras.optimizers import Adam

from keras.callbacks import ModelCheckpoint, TensorBoard, Callback, EarlyStopping

In [25]:
def remove_numbers(ing):
    # remove numbers from ingredients
    
    return [[re.sub("\d+", "", x) for x in y] for y in ing]

    
def remove_special_chars(ing):
    # remove certain special characters from ingredients
   
    ing = [[x.replace("-", " ") for x in y] for y in ing] 
    ing = [[x.replace("&", " ") for x in y] for y in ing] 
    ing = [[x.replace("'", " ") for x in y] for y in ing] 
    ing = [[x.replace("''", " ") for x in y] for y in ing] 
    ing = [[x.replace("%", " ") for x in y] for y in ing] 
    ing = [[x.replace("!", " ") for x in y] for y in ing] 
    ing = [[x.replace("(", " ") for x in y] for y in ing] 
    ing = [[x.replace(")", " ") for x in y] for y in ing] 
    ing = [[x.replace("/", " ") for x in y] for y in ing] 
    ing = [[x.replace("/", " ") for x in y] for y in ing] 
    ing = [[x.replace(",", " ") for x in y] for y in ing] 
    ing = [[x.replace(".", " ") for x in y] for y in ing] 
    ing = [[x.replace(u"\u2122", " ") for x in y] for y in ing] 
    ing = [[x.replace(u"\u00AE", " ") for x in y] for y in ing] 
    ing = [[x.replace(u"\u2019", " ") for x in y] for y in ing] 

    return ing
    
    
def make_lowercase(ing):
    # make all letters lowercase for all ingredients
    
    return [[x.lower() for x in y] for y in ing]
    
    
def remove_extra_whitespace(ing):
    # removes extra whitespaces
    
    return [[re.sub( '\s+', ' ', x).strip() for x in y] for y in ing] 
    
    
def stem_words(ing):
    # word stemming for ingredients
    
    lmtzr = WordNetLemmatizer()
    
    def word_by_word(strng):
        
        return " ".join(["".join(lmtzr.lemmatize(w)) for w in strng.split()])
    
    return [[word_by_word(x) for x in y] for y in ing] 
    
    
def remove_units(ing):
    # remove certain words from ingredients
    
    remove_list = ['g', 'lb', 's', 'n']
        
    def check_word(strng):
        
        s = strng.split()
        resw  = [word for word in s if word.lower() not in remove_list]
        
        return ' '.join(resw)

    return [[check_word(x) for x in y] for y in ing] 


In [26]:
### by all the words

df=pd.read_json('./train.json')
X = df['ingredients'].values
Y = df['cuisine'].values

In [27]:
X = make_lowercase(X)           # 소문자로 변환
X = remove_numbers(X)           # 숫자 제거 
X = remove_special_chars(X)     # 특수 문자제거
X = remove_extra_whitespace(X)  # 추가 공백 제거
X = remove_units(X)             # ['g', 'lb', 's', 'n'] 와 같은 단위 제거
# X = stem_words(X)               # Lemmatization(원형화) nltk를 활용한 WordNetLemmatizer

In [28]:
df['ingredients_preprocessed'] = X

In [29]:
df['ingredients_as_sentence'] = df['ingredients_preprocessed'].apply(', '.join)

In [30]:
uniques = list(set([item for sublist in X for item in sublist]))
print(len(uniques))

6686


In [31]:
X = df['ingredients_as_sentence'].values

In [32]:
for ing in uniques:
    temp = ing.split(' ')
    for word in temp:
        if word not in uniques:
            uniques.append(word)
print(len(uniques))


8524


In [33]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.2 ,random_state=2019, stratify = Y)
print(X_train.shape, X_test.shape, y_train.shape, y_test.shape)

(31819,) (7955,) (31819,) (7955,)


In [34]:
print(X_train[0])

bourbon whiskey, water, simple syrup, granulated sugar, fresh mint, powdered sugar, mint sprigs


In [35]:
EMBEDDING_FILE =  './crawl-300d-2M.vec'

def load_embeddings(filename):
    embeddings = {}
    with open(filename) as f:
        for line in f:
            values = line.rstrip().split(' ')
            word = values[0]
            vector = np.asarray(values[1:], dtype='float32')
            embeddings[word] = vector
    return embeddings

embeddings = load_embeddings(EMBEDDING_FILE)

In [36]:
x = np.r_[X_train, X_test]
### movie tokenizer
tokenizer = Tokenizer(lower=True, filters='\n\t')
tokenizer.fit_on_texts(x)
x_train = tokenizer.texts_to_sequences(X_train)
x_test  = tokenizer.texts_to_sequences(X_test)
vocab_size = len(tokenizer.word_index) + 1  # +1 is for zero padding.
print('vocabulary size: {}'.format(vocab_size))

vocabulary size: 4147


In [39]:
from keras.preprocessing import text, sequence

maxlen = len(max((s for s in np.r_[x_train, x_test]), key=len))
x_train = sequence.pad_sequences(x_train, maxlen=maxlen, padding='post')
x_test = sequence.pad_sequences(x_test, maxlen=maxlen, padding='post')
print('maxlen: {}'.format(maxlen))
print(x_train.shape)
print(x_test.shape)

maxlen: 141
(31819, 141)
(7955, 141)


In [40]:
def filter_embeddings(embeddings, word_index, vocab_size, dim=300):
    embedding_matrix = np.zeros([vocab_size, dim])
    for word, i in word_index.items():
        if i >= vocab_size:
            continue
        vector = embeddings.get(word)
        if vector is not None:
            embedding_matrix[i] = vector
    return embedding_matrix

embedding_size = 300
embedding_matrix = filter_embeddings(embeddings, tokenizer.word_index,
                                     vocab_size, embedding_size)
print('OOV: {}'.format(len(set(tokenizer.word_index) - set(embeddings))))

OOV: 1775


In [41]:
class Attention(Layer):
    """
    Keras Layer that implements an Attention mechanism for temporal data.
    Supports Masking.
    Follows the work of Raffel et al. [https://arxiv.org/abs/1512.08756]
    # Input shape
        3D tensor with shape: `(samples, steps, features)`.
    # Output shape
        2D tensor with shape: `(samples, features)`.
    :param kwargs:
    Just put it on top of an RNN Layer (GRU/LSTM/SimpleRNN) with return_sequences=True.
    The dimensions are inferred based on the output shape of the RNN.
    Example:
        model.add(LSTM(64, return_sequences=True))
        model.add(Attention())
    """
    def __init__(self, step_dim,
                 W_regularizer=None, b_regularizer=None,
                 W_constraint=None, b_constraint=None,
                 bias=True, **kwargs):
        self.supports_masking = True
        self.init = initializers.get('glorot_uniform')

        self.W_regularizer = regularizers.get(W_regularizer)
        self.b_regularizer = regularizers.get(b_regularizer)

        self.W_constraint = constraints.get(W_constraint)
        self.b_constraint = constraints.get(b_constraint)

        self.bias = bias
        self.step_dim = step_dim
        self.features_dim = 0
        super(Attention, self).__init__(**kwargs)

    def build(self, input_shape):
        assert len(input_shape) == 3

        self.W = self.add_weight((input_shape[-1],),
                                 initializer=self.init,
                                 name='{}_W'.format(self.name),
                                 regularizer=self.W_regularizer,
                                 constraint=self.W_constraint)
        self.features_dim = input_shape[-1]

        if self.bias:
            self.b = self.add_weight((input_shape[1],),
                                     initializer='zero',
                                     name='{}_b'.format(self.name),
                                     regularizer=self.b_regularizer,
                                     constraint=self.b_constraint)
        else:
            self.b = None
        self.built = True

    def compute_mask(self, input, input_mask=None):
        return None

    def call(self, x, mask=None):
        features_dim = self.features_dim
        step_dim = self.step_dim
        eij = K.reshape(K.dot(K.reshape(x, (-1, features_dim)),
                        K.reshape(self.W, (features_dim, 1))), (-1, step_dim))
        if self.bias:
            eij += self.b
        eij = K.tanh(eij)
        a = K.exp(eij)
        if mask is not None:
            a *= K.cast(mask, K.floatx())
        a /= K.cast(K.sum(a, axis=1, keepdims=True) + K.epsilon(), K.floatx())
        a = K.expand_dims(a)
        weighted_input = x * a
        return K.sum(weighted_input, axis=1)

    def compute_output_shape(self, input_shape):
        return input_shape[0],  self.features_dim

In [50]:
def build_model(maxlen, vocab_size, embedding_size, embedding_matrix):
    input_words = Input((maxlen, ))
    x_words = Embedding(vocab_size,
                        embedding_size,
                        weights=[embedding_matrix],
                        mask_zero=True,
                        trainable=False)(input_words)
    x_words = SpatialDropout1D(0.3)(x_words)
    x_words = Bidirectional(LSTM(50, return_sequences=True))(x_words)
    x = Attention(maxlen)(x_words)
    x = Dropout(0.2)(x)
    x = Dense(50, activation='relu')(x)
    x = Dropout(0.2)(x)
    pred = Dense(20, activation='softmax')(x)

    model = Model(inputs=input_words, outputs=pred)
    return model


In [44]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
y_trainn = le.fit_transform(y_train)

In [45]:
print(y_trainn[0])

16


In [54]:
model = build_model(maxlen, vocab_size, embedding_size, embedding_matrix)
model.compile(optimizer='nadam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
model.summary()

save_file = './model_attention.h5'
history = model.fit(x_train, y_trainn,
                    epochs=200, verbose=1,
                    batch_size=512, shuffle=True)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_6 (InputLayer)         (None, 141)               0         
_________________________________________________________________
embedding_6 (Embedding)      (None, 141, 300)          1244100   
_________________________________________________________________
spatial_dropout1d_6 (Spatial (None, 141, 300)          0         
_________________________________________________________________
bidirectional_6 (Bidirection (None, 141, 100)          140400    
_________________________________________________________________
attention_6 (Attention)      (None, 100)               241       
_________________________________________________________________
dropout_11 (Dropout)         (None, 100)               0         
_________________________________________________________________
dense_11 (Dense)             (None, 50)                5050      
__________

31819/31819 [==============================] - 79s 2ms/step - loss: 0.7276 - acc: 0.7728
Epoch 67/200
31819/31819 [==============================] - 81s 3ms/step - loss: 0.7272 - acc: 0.7725
Epoch 68/200
31819/31819 [==============================] - 81s 3ms/step - loss: 0.7214 - acc: 0.7724
Epoch 69/200
31819/31819 [==============================] - 82s 3ms/step - loss: 0.7202 - acc: 0.7735
Epoch 70/200
31819/31819 [==============================] - 81s 3ms/step - loss: 0.7207 - acc: 0.7741
Epoch 71/200
31819/31819 [==============================] - 83s 3ms/step - loss: 0.7106 - acc: 0.7756
Epoch 72/200
31819/31819 [==============================] - 80s 3ms/step - loss: 0.7115 - acc: 0.7771
Epoch 73/200
31819/31819 [==============================] - 80s 3ms/step - loss: 0.7083 - acc: 0.7779
Epoch 74/200
31819/31819 [==============================] - 82s 3ms/step - loss: 0.7080 - acc: 0.7771
Epoch 75/200
31819/31819 [==============================] - 82s 3ms/step - loss: 0.7104 - acc: 

Epoch 146/200
31819/31819 [==============================] - 83s 3ms/step - loss: 0.5965 - acc: 0.8079
Epoch 147/200
31819/31819 [==============================] - 83s 3ms/step - loss: 0.5987 - acc: 0.8080
Epoch 148/200
31819/31819 [==============================] - 81s 3ms/step - loss: 0.5979 - acc: 0.8047
Epoch 149/200
31819/31819 [==============================] - 80s 3ms/step - loss: 0.5892 - acc: 0.8106
Epoch 150/200
31819/31819 [==============================] - 83s 3ms/step - loss: 0.5940 - acc: 0.8066
Epoch 151/200
31819/31819 [==============================] - 80s 3ms/step - loss: 0.5844 - acc: 0.8122
Epoch 152/200
31819/31819 [==============================] - 82s 3ms/step - loss: 0.5950 - acc: 0.8094
Epoch 153/200
31819/31819 [==============================] - 83s 3ms/step - loss: 0.5911 - acc: 0.8079
Epoch 154/200
31819/31819 [==============================] - 81s 3ms/step - loss: 0.5849 - acc: 0.8123
Epoch 155/200
31819/31819 [==============================] - 81s 3ms/step

In [55]:
y_pred = model.predict(x_test, batch_size=1024)
y_pred = y_pred.argmax(axis=1).astype(int)
y_pred.shape

(7955,)

In [56]:
y_test = Lec.fit_transform(y_test)

In [57]:
import sklearn
from sklearn.metrics import classification_report

target_names = ['brazilian', 'british', 'cajun_creole', 'chinese', 'filipino', 'french', 'greek',
           'indian', 'irish', 'italian', 'jamaican', 'japanese', 'korean', 'mexican', 'moroccan',
           'russian', 'southern_us', 'spanish', 'thai', 'vietnamese']

score = sklearn.metrics.accuracy_score(y_test, y_pred)

print(score)
print(classification_report(y_test, y_pred ,target_names=target_names))

0.6898805782526712
              precision    recall  f1-score   support

   brazilian       0.48      0.40      0.44        93
     british       0.35      0.30      0.32       161
cajun_creole       0.69      0.60      0.64       309
     chinese       0.72      0.78      0.75       535
    filipino       0.55      0.46      0.50       151
      french       0.51      0.54      0.52       529
       greek       0.65      0.64      0.65       235
      indian       0.80      0.83      0.81       601
       irish       0.46      0.31      0.37       133
     italian       0.78      0.80      0.79      1568
    jamaican       0.52      0.43      0.47       105
    japanese       0.67      0.58      0.62       284
      korean       0.66      0.63      0.64       166
     mexican       0.85      0.85      0.85      1288
    moroccan       0.64      0.62      0.63       164
     russian       0.35      0.27      0.30        98
 southern_us       0.57      0.69      0.63       864
     spa